In [ ]:
# take original rolled daily data and create monthly avearages

#DESC
Dlvod_roll['ID']=Dlvod_roll.index
#lvod_roll[0:10]
#make a copy of ID
ids = Dlvod_roll.loc[:, ['ID']]
#drop id from the db
Dlvod_roll.drop(["ID"], axis = 1, inplace = True) 
#then do date time magic
#print(lvod_roll)
Dlvod_roll.columns = pd.to_datetime(Dlvod_roll.columns)
Dlvod_month_means = Dlvod_roll.resample('M', axis=1).mean()
Dlvod_month_min = Dlvod_roll.resample('M', axis=1).min()
Dlvod_month_med = Dlvod_roll.resample('M', axis=1).median()
Dlvod_month_std = Dlvod_roll.resample('M', axis=1).std()
Dlvod_month_count = Dlvod_roll.resample('M', axis=1).count()
#print(lvod_month_means)

#then add id back in IDs- 
#lvod_roll2 = pd.concat([ids, lvod_roll])
Dlvod_month_meansID = pd.concat([ids, Dlvod_month_means], axis=1,sort=False)
Dlvod_month_medID = pd.concat([ids, Dlvod_month_med], axis=1,sort=False)
Dlvod_month_stdID = pd.concat([ids, Dlvod_month_std], axis=1,sort=False)
Dlvod_month_ctID = pd.concat([ids, Dlvod_month_count], axis=1,sort=False)
Dlvod_month_minID = pd.concat([ids, Dlvod_month_min], axis=1,sort=False)
#delete empty rows before exporting-- this facilitates mapping in arcGIS
# drop a column if the NaN count passes 101
# also for CSV column headings have to be alpha numeric! must change in excel...
Dlvod_month_meansIDno0s = Dlvod_month_meansID.dropna(thresh=(len(Dlvod_month_meansID.columns) - 101))
#Dlvod_month_meansIDno0s.to_csv("Variables/FromPython/CONUS_DescLVODMonthlyMeansNO0x.csv")
Dlvod_month_minsIDno0s = Dlvod_month_minID.dropna(thresh=(len(Dlvod_month_minID.columns) - 101))

#select Growing season only (exlude 4 winter months)
# Drop winter months, had problems establishing multiple ranges- so did this in a truly hokey manner
Dlvod_month_meansGSno0s = Dlvod_month_meansIDno0s.drop(Dlvod_month_meansIDno0s.columns[[1, 2, 11, 12, 13, 14, 23, 24, 25, 26, 35, 36, 37, 38, 47, 48, 49, 50, 59, 60, 61, 62, 71, 72, 73, 74, 83, 84, 85, 86, 95, 96, 97, 98]], axis=1)
Dlvod_month_minsGSno0s = Dlvod_month_minsIDno0s.drop(Dlvod_month_minsIDno0s.columns[[1, 2, 11, 12, 13, 14, 23, 24, 25, 26, 35, 36, 37, 38, 47, 48, 49, 50, 59, 60, 61, 62, 71, 72, 73, 74, 83, 84, 85, 86, 95, 96, 97, 98]], axis=1)

#write to cvs
Dlvod_month_meansGSno0s.to_csv("Variables/FromPython/CONUS_DescLVODMonthlyMeansGSNO0x.csv")
Dlvod_month_minsGSno0s.to_csv("Variables/FromPython/CONUS_DescLVODMonthlyMINsGSNO0x.csv")



#************************************8
###RELATIVIZE DESCENDING by 2010-2017 averages  for maps (?)

#divide the values by the annual average... to get relative LVOD (and exclude biomass values)
#caculate over allmean then divide entire monthly DESC and ASC datasets by that

#first establish index
#Dlvod_month_meanGSno0s['ID']=Dlvod_month_meanGSno0s.index
#Dlvod_month_minGSno0s['ID']=Dlvod_month_minGSno0s.index

#for min- make asingle column of the average values  #df.iloc[:, [2,5,6,7,8]].mean(axis=1)
Dlvod_month_minGSnoID = Dlvod_month_minsGSno0s[Dlvod_month_minsGSno0s.columns[1:69]]
Dlvod_20102017AvgMin = Dlvod_month_minGSnoID.mean(axis=1)

#for avg make asingle column of the average values
Dlvod_month_meanGSnoID = Dlvod_month_meanGSno0s[Dlvod_month_minGSno0s.columns[1:69]]
Dlvod_20102017AvgAvg = Dlvod_month_meanGSnoID.mean(axis=1)

#divide by overall average  (SHOULD I USE THIS?  I'm thinking averages by month is a better idea-- all Mays...)
# df1.div(df2.iloc[0], axis='columns')
Dlvod_month_RELmeanGSno0s = Dlvod_month_meanGSno0s.divide(Dlvod_20102017AvgAvg.iloc[0], axis='columns')
Dlvod_month_RELminnGSno0s = Dlvod_month_minGSno0s.divide(Dlvod_20102017AvgMin.iloc[0], axis='columns')
Dlvod_month_RELmeanGSno0s.reset_index(inplace=True)
Dlvod_month_RELminnGSno0s.reset_index(inplace=True)
#Dlvod_month_RELminnGSID = Dlvod_month_RELminnGSno0s.rename_axis('ID').reset_index()
#Dlvod_month_RELminnGSno0s["ID2"] = Dlvod_month_RELminnGSno0s.index
Dlvod_month_RELmeanGSno0s.drop('ID', axis=1, inplace=True)
Dlvod_month_RELminnGSno0s.drop('ID', axis=1, inplace=True)
Dlvod_month_RELmeanGS = Dlvod_month_RELmeanGSno0s.rename({'index':'ID'}, axis=1) 
Dlvod_month_RELminGS = Dlvod_month_RELminnGSno0s.rename({'index':'ID'}, axis=1) 

#write to cvs
Dlvod_month_RELmeanGS.to_csv("Variables/FromPython/CONUS_DescLVODMonthlyRELMeansGSNO0x.csv")
Dlvod_month_RELminGS.to_csv("Variables/FromPython/CONUS_DescLVODMonthlyRELMINsGSNO0x.csv")

#next in excel- sum values and delete all rows with 100% zeros, then delete that column, Also replace headers with text-dates,
#then save as old excel to join to arc

#CALC diff  this needs to be indexed on ID first-- DESC and ASC have different # of rows
# Monthlydiff2 = Alvod_month_meansID - Dlvod_month_meansID